In [2]:
import numpy as np
import cv2
import os
from tqdm.notebook import tqdm
import random
import nbimporter
from Intergral import integral_image, integral_of_squares

In [9]:
# def nms(detections, iou_thresh=0.3):

#     if len(detections) == 0:
#         return []

#     # Chuyển về dạng numpy để tính toán nhanh hơn
#     boxes = np.array([[x, y, x + w, y + h, score] for (x, y, w, h, score) in detections])
#     x1, y1, x2, y2, scores = boxes[:,0], boxes[:,1], boxes[:,2], boxes[:,3], boxes[:,4]

#     areas = (x2 - x1 + 1) * (y2 - y1 + 1)
#     order = scores.argsort()[::-1]  # sắp xếp theo score giảm dần

#     keep = []
#     while order.size > 0:
#         i = order[0]
#         keep.append(i)

#         # Tính IoU giữa box i với các box còn lại
#         xx1 = np.maximum(x1[i], x1[order[1:]])
#         yy1 = np.maximum(y1[i], y1[order[1:]])
#         xx2 = np.minimum(x2[i], x2[order[1:]])
#         yy2 = np.minimum(y2[i], y2[order[1:]])

#         w = np.maximum(0.0, xx2 - xx1 + 1)
#         h = np.maximum(0.0, yy2 - yy1 + 1)
#         inter = w * h
#         iou = inter / (areas[i] + areas[order[1:]] - inter)

#         # Giữ lại box có IoU < ngưỡng
#         inds = np.where(iou <= iou_thresh)[0]
#         order = order[inds + 1]

#     # Trả về danh sách (x, y, x2, y2, score)
#     return boxes[keep].tolist()

def nms(detections, window_size=12):
    filtered = []

    for face in detections:
        x, y, w, h, score = face
        merged = False

        for i, existing in enumerate(filtered):
            ex, ey, ew, eh, escore = existing

            # Nếu 2 khung gần nhau (tâm gần nhau)
            if abs(x - ex) < window_size and abs(y - ey) < window_size:
                # Giữ lại khung có score cao hơn
                if score > escore:
                    filtered[i] = face
                merged = True
                break

        if not merged:
            filtered.append(face)

    final_boxes = [(x, y, x + w, y + h, score) for (x, y, w, h, score) in filtered]
    return final_boxes

In [8]:
def get_subwindow_integral(x, y, w, h, gray_image, global_ii=None, global_sii=None):
    """
    Returns: (ii, sii, offset_x, offset_y)
    """
    if global_ii is None:
        global_ii = integral_image(gray_image)
    if global_sii is None:
        global_sii = integral_of_squares(gray_image)

    return global_ii, global_sii, x, y

In [5]:
def load_images(folder, size=(24, 24)):
    images = []
    supported_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.pgm', '.tif')

    for filename in tqdm(os.listdir(folder), desc=f"Loading {folder}"):
        if filename.lower().endswith(supported_exts):
            path = os.path.join(folder, filename)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, size)
            images.append(img)
    return images

In [6]:
def generate_non_face_patches(input_folder, output_folder, patch_size=24, patches_per_image=50):
    
    os.makedirs(output_folder, exist_ok=True)
    supported_exts = ('.jpg', '.jpeg', '.png', '.bmp', '.tif')

    counter = 0
    for filename in tqdm(os.listdir(input_folder), desc="Generating non-face patches"):
        if not filename.lower().endswith(supported_exts):
            continue

        path = os.path.join(input_folder, filename)
        img = cv2.imread(path)
        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        h, w = gray.shape
        if h < patch_size or w < patch_size:
            continue  # ảnh nhỏ quá, bỏ qua

        # Tạo danh sách vị trí đã cắt để tránh trùng
        used_positions = set()

        num_patches = 0
        attempts = 0
        max_attempts = patches_per_image * 10  # tránh lặp vô hạn

        while num_patches < patches_per_image and attempts < max_attempts:
            attempts += 1
            x = random.randint(0, w - patch_size)
            y = random.randint(0, h - patch_size)
            # kiểm tra trùng vị trí (theo bước pixel)
            if (x, y) in used_positions:
                continue
            used_positions.add((x, y))

            patch = gray[y:y + patch_size, x:x + patch_size]
            save_path = os.path.join(output_folder, f"neg_{counter:05d}.png")
            cv2.imwrite(save_path, patch)
            counter += 1
            num_patches += 1